<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Потоки исполнения](#toc1_)    
  - [Поток исполнения](#toc1_1_)    
  - [Потоки и процессы](#toc1_2_)    
  - [Стек потока](#toc1_3_)    
  - [Многопоточность](#toc1_4_)    
  - [Переключение между потоками](#toc1_5_)    
    - [Почему можно сохранять не все регистры](#toc1_5_1_)    
  - [Переключение потоков](#toc1_6_)    
  - [Создание нового потока](#toc1_7_)    
  - [Начальный контекст](#toc1_8_)    
  - [Пример:](#toc1_9_)    
  - [Кооперативная многозадачность](#toc1_10_)    
  - [Вытесняющая многозадачность](#toc1_11_)    
    - [Снова о прерываниях](#toc1_11_1_)    
  - [Таймер](#toc1_12_)    
  - [Планирование потоков](#toc1_13_)    
  - [Простое планирование](#toc1_14_)    
  - [Пропускная способность](#toc1_15_)    
  - [Среднее время ожидания](#toc1_16_)    
  - [Динамическое создание задач](#toc1_17_)    
  - [IO операции](#toc1_18_)    
  - [Утилизация](#toc1_19_)    
  - [Информация о задаче](#toc1_20_)    
  - [IO-bounded и CPU-bounded](#toc1_21_)    
- [Round Robin](#toc2_)    
  - [Достоинства Round Robin](#toc2_1_)    
  - [Round Robin](#toc2_2_)    
  - [Выбор кванта времени](#toc2_3_)    
  - [Планировщик Windows](#toc2_4_)    
    - [Priority Boost](#toc2_4_1_)    
  - [Планировщик Linux (один из)](#toc2_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Потоки исполнения](#toc0_)

## <a id='toc1_1_'></a>[Поток исполнения](#toc0_)

Поток исполнения - это код и его состояние (a. k. a.контекст)

    код - набор инструкций в памяти, на который указывает регистр RIP (указатель команд - указ. какую команду исполнить след.);
    контекст потока включает значения регистров и память

## <a id='toc1_2_'></a>[Потоки и процессы](#toc0_)

Поток работает в контексте некоторого процесса
     
     т. е. поток "живет" в логическом адресном пространстве процесса (пользуется только его памятью);
     несколько потоков могут работать в рамках одного процесса (разделяют память);
     процесс имеет как минимум один поток.

## <a id='toc1_3_'></a>[Стек потока](#toc0_)

Каждый поток исполнения имеет свой собственный стек (память одна, стек свой у каждого)

    стек хранит адреса возвратов и локальные переменные (их невозможно разделить между потоками, кому какую, поэтому у каждого свой);
    для процессора стек - место в памяти, куда указывает RSP (х86).

## <a id='toc1_4_'></a>[Многопоточность](#toc0_)

В системе могут одновременно работать несколько потоков исполнения. Варианты:

    на нескольких ядрах процессора;
    на одном ядре, создавая иллюзию одновременной работы*.

*) задачи разделяются на подзадачи и выполняются "вперемешку".


Пример: 

Иногда многопоточность используют как способ ускорения расчетов. Мы не будем подробнее на этом останавливаться в рамках курса, но эта задача позволит вам познакомиться с возможными пределами такого ускорения.

Пусть у вас есть некоторая задача, которая состоит из 2 частей. Первая часть - инциализация, которую нельзя выполнять в несколько потоков. Время выполнения инициализации занимает 10 секунд (Tinit=10). Вторая часть - вычисления, которые можно выполнять многопоточно, причем время выполнения этой части i потоками параллельно занимает 10/i секунд (T(i,calc)=10/i) - другими словами, чем больше потоков вы создадите, тем быстрее вы сможете завершить вычисления.

Внимание вопрос: найдите минимальное время в секундах, за которое можно завершить задачу (и инициализацию, и вычисления), при условии, что количество потоков, которые вы можете создать, не ограничено.

Более формально: найти предел lim⁡i→∞(Tinit+T(i,calc)). Очевидно он равен 10. Т.е. производительность определяется временем инициализации.


## <a id='toc1_5_'></a>[Переключение между потоками](#toc0_)

Для переключения между потоками необходимо:

    сохранить контекст исполняемого потока;
    восстановить контекст потока, на который мы переключаемся.

Пример переключалки/сохранялки контекста на x86 (switch.S):

        .text
        .code64
        .global __switch_threads

    __switch_threads:
        pushq %rbx              // регистры (контекст) сохраняется на стек
        pushq %rbp              // только необходмые регистры общего назначения
        pushq %r12
        pushq %r13
        pushq %r14
        pushq %r15
        pushfq                  // флаговый регистр
        
        /* rdi - первый аргумент функции (**prev указатель на указатель) */
        movq %rsp, (%rdi)       // текущий указатель стека процесса (rsp) сохраняется через rdi туда, куда указал **prev
                                // () - разыменование **prev до *prev, копирование в адрес *prev значения регистра rsp

        /* rsi - второй аргумент функции (*next) */
        movq %rsi, %rsp         // указатель стека rsp теперь хранит значение адреса, переданного во втором аргументе
                                // теперь для процессора стек - это область памяти по новому адресу, где должен лежать контекст 
                                // нового процесса

        popfq                   // восстановить регистры (контекст) для процесса, на который переключаемся
        popq %r15
        popq %r14
        popq %r13
        popq %r12
        popq %rbp
        popq %rbx

        retq                    // передаем управление назад (функция retq достает адрес возврата со стека)
                                // теперь уже "нового" стека, потока, на который переключаемся

C API этой переключалки

    void switch_threads(void ∗∗prev, void ∗next);
    по завершении функции ∗prev (поток, с которого переключаемся) будет указывать на сохраненный контекст;
    next (поток, на который переключаемся) указывает на сохраненный контекст потока, накоторый мы переключаемся.


### <a id='toc1_5_1_'></a>[Почему можно сохранять не все регистры](#toc0_)

Пусть есть функция А, которая вызывает функцию Б и потом продолжает работу. Для работы ей нужны регистры, возможно, они нужны также и после выполнения функции Б до завершения функции А. Кто должен сохранять регистры?

    A:
        ...
        ...
        call B
        ...
        ...
        retq

Соглашение в х86-64 состоит в том, что вызывающий функцию код ответственнен за сохранение ВСЕХ ИЗМЕНЯЕМЫХ регистров. Соответственно вызываемый код ответственнен за сохранение только тех регистров, которые изменяет в ходе своей работы он.  
В данном случае А должна сохранить изменяемые регистры перед вызовом Б, Б выполнит свою работу, сохранив те регистры, которые изменяет он и перед возвратом восстановив их.

В __switch_threads мы полагается на ABI (конкретно System V), согласно ABI компилятор должен сохранить остальные регистры, если они ему дальше понадобятся (правда это касается только регистров общего назначения, есть и другие).  

*) Регистры специльного назначения переключаются точно также как и регистры общего назначения - просто берем и записываем туда значения, которые вам нужны.

## <a id='toc1_6_'></a>[Переключение потоков](#toc0_)

        Tread0            Tread1
        |                 .
    CALL|switch_thr       .  
        --------> retq -> |                         // retq функции switch_thr
        .                 |
        .             CALL|switch_thr     
     RET|<- retq <---------                         // retq функции switch_thr в точку возврата RET потока Thread0
        |                 .
        |                 .
    
С т.з. Thread0 просто произошел вызов функци switch_thr и возврат из нее в точку RET.


## <a id='toc1_7_'></a>[Создание нового потока](#toc0_)

Как создать новый поток и переключиться на него впервый раз?
    
    нам нужно выделить место для хранения указателя на контекст;
    нам нужно выделить место под стек нового потока;
    нам нужно сохранить на стеке начальный контекст и сохранить указатель на него.

## <a id='toc1_8_'></a>[Начальный контекст](#toc0_)

    struct switch_frame {
        uint64_t rflags;            // флаги вначале (меньшие значения адреса)
        uint64_t r15;
        uint64_t r14;
        uint64_t r13;
        uint64_t r12;
        uint64_t rbp;
        uint64_t rbx;               // rbx в конце (большие значения адреса)
        uint64_t rip;               // поле для адреса возврата, который должен храниться непосредственно перед стеком
    } __attribute__((packed));

Порядок в х86 обратный, т.к. стек перевернутый от Х до 0, т.е. более поздние сохранения на стек сохраняются по меньшим физическим адресам.  

Поле rip нужно, т.к. поток еще не вызывался и из функции switch_frame пока возвращаться некуда. Что же сюда записать?   
Сюда заносится адрес кода (функции), с которого начнется исполнение потока (точка входа в поток). 

## <a id='toc1_9_'></a>[Пример:](#toc0_)

Многопоточность можно организовать на уровне приложения, без задействования возможностей ОС, для которой это будет один поток.  

	struct thread {	void *context; }; 			// индентификатор/дескриптор потока (просто общий адрес)

	void switch_threads(struct thread *from, struct thread *to) {		// обертка для вызова __switch_threads
		void __switch_threads(void **prev, void *next);			// соответствующий объектник подключается в make файл:
		__switch_threads(&from->context, to->context); 			// example: switch.o main.o
	}									// 	$(CC) -g $^ -o $@

	struct thread *__create_thread(size_t stack_size, void (*entry)(void)) {// создает стек
		const size_t size = stack_size + sizeof(struct thread);
		struct switch_frame frame;					// создаем на стеке шаблон для стека
		struct thread *thread = malloc(size);				// аллокация под структуру
		if (!thread)							// если не аллоцировалось
			return thread;

		memset(&frame, 0, sizeof(frame));				// начальный контекст инициализируется 0
		frame.rip = (uint64_t)entry;					// кроме поля адреса возврата (входа)
		
		thread->context = (char *)thread + size - sizeof(frame);
		memcpy(thread->context, &frame, sizeof(frame));	// копируем инициализированный начальный контекст в дин.память
		return thread;							// возвращаем указатель на него
	}

	static struct thread _thread0;			// структура под исполнение функции main() - главный поток, на самом деле не нужно
	static struct thread *thread[3];		// стат.массив указателей на дескрипторы потоков (точки входа (они же тела потоков))


	static void thread_entry1(void)			// точка входа - просто фукнция (без циклов и всякого такого)
	{
		printf("In thread1, switching to thread2...\n");
		switch_threads(thread[1], thread[2]);	// может вызывать переключение на другой поток
		printf("Back in thread1, switching to thread2...\n");
		switch_threads(thread[1], thread[2]);
		printf("Back in thread1, switching to thread0...\n");
		switch_threads(thread[1], thread[0]);	// в конце - возврат в главный поток
	}
	...

	int main(void)
	{
		thread[0] = &_thread0;				// главный поток main()
		thread[1] = create_thread(&thread_entry1);
		thread[2] = create_thread(&thread_entry2);

		printf("In thread0, switching to thread1...\n");
		switch_threads(thread[0], thread[1]);		// уходим из главного в 1
		printf("Retunred to thread 0\n");		// когда вернулись - финиш

		destroy_thread(thread[2]);
		destroy_thread(thread[1]);

		return 0;
	}

	$ ./example
	In thread0, switching to thread1...
	In thread1, switching to thread2...
	In thread2, switching to thread1...
	Back in thread1, switching to thread2...
	Back in thread2, switching to thread1...
	Back in thread1, switching to thread0...
	Retunred to thread 0

## <a id='toc1_10_'></a>[Кооперативная многозадачность](#toc0_)

Невытесняющая (кооперативная) многозадачность (пример выше)

    поток должен сам вызвать функцию переключения;
    что если в коде содержится ошибка?
    или мы обращаемся к библиотеке, которая выполняетдолгую операцию?

## <a id='toc1_11_'></a>[Вытесняющая многозадачность](#toc0_)

Вытесняющая (preemptive) многозадачность

    поток снимается ОС с CPU "силой", по истечении кванта времени;
    синхронизация потоков при этом усложняется;
    как организовать вытесняющую многозадачность?

### <a id='toc1_11_1_'></a>[Снова о прерываниях](#toc0_)

Обработчик прерывания "прерывает" исполняемый код
    
    но обработчик работает в контексте прерванного потока;
    функцию переключения контекста можно вызвать от имени потока из обработчика прерываний.



            Tread0                            Tread1
            |                                 .
            |interrupt       Int_Handler0     .                                 // вызывается обработчик прерывания
            ---------------->|                .        
            .                |                .
            .            CALL|switch_thr      .                                 // который выз. переключение потока
            .                --------> retq ->|
            .                .                |interrupt       Int_Handler1     // какое-то другое прерывание
            .                .                ---------------->|
            .                .                .            CALL|switch_thr      // передает управление в первый обработчик
            .             RET|<- retq <-------------------------      
        RET|<- iretq <--------                .                .                            
            |                .                .                .   
            |                .                .                .    
                                      (iretq)->                                 // когда-то от какого-то обработчика прерываний 
                                              |                                 // может вернуться сюда (всегда парами interrupt / iretq)

Принудительность переключения потока обеспечивается механизмом прерываний, который не зависит от пользовательского кода.  

Источником прерываний может быть, например таймер, или любое другое устройство.  

Пользовательскому коду организовать работу с прерываниями сложно. В Linux можно использовать сигналы, например таймер alarm().  

    Тогда потоки, это функции в бесконечном цикле. 
    Переключатель потока - такой же как был.  
    Регистрируется функция обработчика прерываний, которая по сигналу вызывает переключение потоков. 
    Главный поток вешается в бесконечный цикл.  


## <a id='toc1_12_'></a>[Таймер](#toc0_)

Таймер может генерировать прерывания с заданной периодичностью

    Programmable Interval Timer (PIT, intel 8253) - IBMPC;  // древний таймер, со времен IBM PC
    High Precision Event Timer (HPET);                      // точная штука, еще назыв. медиатаймер
    Local APIC Timer.                                       // также присутстсует обычно на х86

## <a id='toc1_13_'></a>[Планирование потоков](#toc0_)

Планировщик (scheduler) - компонент ОС, который определяет
    когда переключаться с потока;
    на какой поток переключаться.

## <a id='toc1_14_'></a>[Простое планирование](#toc0_)

Рассмотрим простейшую задачу планирования

    все задачи известны заранее;
    про каждую задачу известно, сколько времени она займет;
    задачи работают без переключений;
    т. е. нам осталось только определить порядок.

## <a id='toc1_15_'></a>[Пропускная способность](#toc0_)

Задачи - время(с):

    A - 9
    B - 7
    C - 3

Суммарное время в любом порядке 19 с. Без учета приоритетности, разбивки задач, времени переключения.

## <a id='toc1_16_'></a>[Среднее время ожидания](#toc0_)

Пусть все задачи принадлежат разным пользователям
    
    пользователю важно, сколько ему нужно ждать завершения его задачи;
    давайте в качестве метрики использовать среднее время ожидания.

В этом случае порядок важен. АBC ~14.6c, CBA ~10.6c. Выгодно по этому критерию короткие задачи ставить вперед.




In [26]:
# В первой строке вам дается число задач N. В следующей строке идет описание задач, для каждой задачи вам дана ее продолжительность - Ti​ (где i - это номер задачи, от 0 до N−1 не включительно). На выход вам требуется вывести номера задач (задачи нумеруются с 0) в порядке, который минимизирует среднее время ожидания завершения задачи, как это было объяснено ранее.

SAMPLES = "3\n9 7 3", 
READER = (x for x in SAMPLES[0].split('\n')); input = lambda: next(READER)

# put your python code here
n = int(input())
D = dict(zip(range(n), list(map(int, input().split()))))
print(*sorted(D, key=lambda x: D.get(x)))              # просто сортировка словаря по ключу

2 1 0


## <a id='toc1_17_'></a>[Динамическое создание задач](#toc0_)

Зачастую все задачи не известны заранее

    задачи могут создаваться в произвольные моменты времени;
    каждая вновь появившаяся задача может изменить решение планировщика.

## <a id='toc1_18_'></a>[IO операции](#toc0_)

Задачи могут давать команды устройствам или ждать каких-то событий:

    запись/чтение на/с HDD (порядка нескольких мс);
    ждать входящих соединений по сети;
    ждать, пока пользователь нажмет на клавишу.

Пока задача ждет завершения IO операции, можно забрать у нее CPU 

    время ожидания может быть большим (1мс - очень много для CPU);
    утилизация CPU - сколько времени CPU делал полезную работу.

## <a id='toc1_19_'></a>[Утилизация](#toc0_)

Варианты утилизации ресурсов CPU.

    1. Делать вставки новых задач конкретно в IO паузы уже выполняющихся задач.
    2. Прерывать начатые задачи для вставки новых задач до соответственно их IO паузы, после которой завершать исходные задачи.

Второй подход значительно увеличивает утилизацию (полезное использоване) ЦПУ. Как минимум в силу случайного характера возникновения новых задач и взаимного "съедания" IO пауз задач.

## <a id='toc1_20_'></a>[Информация о задаче](#toc0_)

Зачастую время работы задачи и расписание ее IO неизвестны

    задачи могут влиять друг на друга или зависеть от внешних обстоятельств;
    мы можем оценивать эти параметры и классифицировать задачи (на основе истории поведения).

## <a id='toc1_21_'></a>[IO-bounded и CPU-bounded](#toc0_)

Простая классификация задач.  

    IO-bounded задачи - много IO, но мало вычислений:
        
        например, текстовый редактор;
        вообще приложения, ожидающие ввода пользователя.

    CPU-bounded задачи - много вычислений, но мало IO:

        например, научные вычисления;
        компиляция программ.



# <a id='toc2_'></a>[Round Robin](#toc0_)

Round Robin - выдаем потокам квант времени на CPU по очереди:

    поток, отработавший свой квант, и не отдавший самостоятельно управление ЦПУ, встает в конец очереди;
    каждый новый поток встает в конец очереди;
    потоки, дождавшиеся завершения IO, встают в конец очереди;
    CPU отдается потоку в начале очереди.

## <a id='toc2_1_'></a>[Достоинства Round Robin](#toc0_)

К списку достоинств Round Robin можно отнести:
    подход очень прост;
    время ожидания CPU ограничено - никто не голодает.

## <a id='toc2_2_'></a>[Round Robin](#toc0_)

см. ./scheduler.cpp

## <a id='toc2_3_'></a>[Выбор кванта времени](#toc0_)

Из каких соображений стоит выбирать квант времени?

    чем больше квант
        тем меньше доля времени на переключение;
        тем больше время отклика;
    чем меньше квант
        тем больше доля времени на переключение;
        тем меньше время отклика

## <a id='toc2_4_'></a>[Планировщик Windows](#toc0_)

Потокам в Windows назначен приоритет

    приоритет состоит из класса и приоритета внутри класса.

На исполнение выбирается поток с наивысшим приоритетом

    для потоков с равными приоритетами используется RR.

*) разделение по приоритетам (32 уровня) делает не совсем "честным" и делает возможной ситуацию, когда низкоприоритетный поток вообще не получает процессорного времени. Чтобы этого избежать ->


### <a id='toc2_4_1_'></a>[Priority Boost](#toc0_)

Чтобы избежать неограниченного голодания потоков, Windows иногда повышает им приоритет:
    
    если поток отвечает за видимую часть UI;
    при получении ввода или завершении операции IO;
    для "случайно" выбранных потоков.

## <a id='toc2_5_'></a>[Планировщик Linux (один из)](#toc0_)

Completely Fair Scheduler (CFS) - честный планировщик:

    для каждого потока поддерживается "виртуальное время";
    "виртуальное время" увеличивается, когда поток работает;
    CPU отдаем потоку с наименьшим "виртуальным временем".

\*) Конкретно в Linux все потоки поддерживаются в виде rb-tree, упоряд. по виртуальному времени, выбирается самый левый узел для выполнения.  
\*\*) Когда поток долго не работал и возвращается в дерево - у остальных виртуальное время может уйти далеко вперед, и они будут вынуждены ждать, пока этот самый левый поток из догонит. С такими случаями планировщик борется различными способами.


    CFS

    chrt -m                                         # основные текущие настройки планировщика
    chrt -p 33719                                   # приоритет и политика планировщика для процесса
    sysctl -A | grep "sched" | grep -v "domain"     # переменные ядра для планировщика задач
    cat /proc/sched_debug                           # все настройки и состояние планировщика

Особенности - нет кванта времени, никаких эвристик по поводу процессов (неуязвим к атакам на такие эвристики), одна основная настройка:
    
    /proc/sys/kernel/sched_min_granularity_ns           -> low values for “desktop” (i.e., low latencies), high - for “server” (i.e., good batching - меньше времени утилизируется на переключения задач) 